## **SETD2 with SAM**

## **Import modules**

In [1]:
import os
import sys

data_path=os.path.abspath('/media/dataHog/hca/SETD2') #Where your group data is
base_path=os.path.abspath('/media/dataHog/hca/SimFound_v2/source/') #Where your source code is (SFv2)

sys.path.append(base_path)
sys.path.append(data_path)

import importlib

import warnings
warnings.filterwarnings('ignore')
import Protocols as P

import main
import tools
import Featurize as F
import Trajectory as T


from simtk.unit import *

## **Define Project**

In [9]:
importlib.reload(main)
importlib.reload(T)

workdir=data_path
results=workdir+'/results'
inputs=workdir+'inputs/structures/'



#protein=['WT', 'R167Q', 'I602G']
protein=['ssK36']

parameters=['310K']
timestep=20*picoseconds

project=main.Project(title='ssK36', 
                     hierarchy=('protein', 'parameter'), 
                     workdir=workdir,
                     parameter=parameters, 
                     replicas=2, 
                     protein=protein,  
                     topology='ssK36_X.pdb',
                     timestep=timestep,
                    initial_replica=1)


project_systems=project.setSystems()

Converted parameter "temperature" (in K) into scalar: 310.0 K
System ssK36-310K-1 with parameter 310.0 K defined
System ssK36-310K-2 with parameter 310.0 K defined


## **Simulation Protocol**

## Set defaults

In [10]:
NPT_equilibration={'ensemble': 'NPT',
                'step': 5*nanoseconds, 
                'report': timestep, 
                'restrained_sets': {'selections': ['protein and backbone'],
                                    'forces': [100*kilojoules_per_mole/angstroms]}}

NPT_production={'ensemble': 'NPT',
                'step': 50*nanoseconds, 
                'report': timestep}
NPT_production2={'ensemble': 'NPT',
                'step': 50*nanoseconds, 
                'report': timestep}

## Setup simulation protocol

In [11]:
importlib.reload(P)
importlib.reload(tools)
#extra_pdb = ['ZNB_open.pdb', 'SAM.pdb']
#smiles = 'C[S+](CC[C@@H](C(=O)[O-])N)C[C@@H]1[C@H]([C@H]([C@@H](O1)N2C=NC3=C(N=CN=C32)N)O)O'
xml_files = ['protein.ff14SB.xml', 'tip4pew.xml']


simulation=P.Protocols(project, overwrite_mode=False)
    
simulation.pdb2omm(input_pdb=project.input_topology,
                                     ff_files=xml_files,
                                     protonate=True,
                                     solvate=True,
                                     pH_protein=7.0,
                                     box_size=6)

simulation.setSimulations(dt=0.002*picoseconds, 
                                  temperature = 310,
                                  pH=7.0,
                                  friction = 1/picosecond,
                                  equilibrations=[NPT_equilibration],
                                  productions=[NPT_production, NPT_production2],
                                  pressure=1*atmospheres)


Setting openMM simulation of ssK36-310K-1 protocols in /media/dataHog/hca/SETD2/ssK36-310K/replicate1
Setting openMM simulation of ssK36-310K-2 protocols in /media/dataHog/hca/SETD2/ssK36-310K/replicate2
	Warning! Found conserved openMM system(s). /media/dataHog/hca/SETD2/ssK36-310K/replicate1/omm_1.pkl
	Status open: 1
	Loading conserved openMM system:  /media/dataHog/hca/SETD2/ssK36-310K/replicate1/omm_1.pkl
	Warning! Found conserved openMM system(s). /media/dataHog/hca/SETD2/ssK36-310K/replicate2/omm_1.pkl
	Status open: 1
	Loading conserved openMM system:  /media/dataHog/hca/SETD2/ssK36-310K/replicate2/omm_1.pkl
{'ssK36-310K-1': <main.System object at 0x7f311d24f290>, 'ssK36-310K-2': <main.System object at 0x7f311d24f890>}
setting up:  ssK36-310K-1
setting up:  ssK36-310K-2


## **WARNING!** Run simulation(s)

In [ ]:

gpu_index='1'
simulation.runSimulations(run_productions=True,  
                              compute_time=4,
                              reportFactor=0.025,
                             gpu_index=gpu_index,
                         minimum_effort=True)   


System:  ssK36-310K-1
	Avaliable platform 1: CPU
	Avaliable platform 2: CUDA
	Avaliable platform 3: OpenCL

Energy minimization
minimization_Emin-1
	A complete checkpoint was found.
	System is complete.
	System final potential energy: -395298.92832956044 kJ/mol (complete)

EQ run 1: NPT
equilibration_NPT-1
	A complete checkpoint was found.
	System is complete.
	System final potential energy: -320540.34189579426 kJ/mol (complete)

Production run 1: NPT
production_NPT-1
	A complete checkpoint was found.
	System is complete.
	System final potential energy: -320107.5687896705 kJ/mol (complete)

Production run 2: NPT
production_NPT-2
	A timeout checkpoint was found.
	Simulation time: 49920.842000000004 ps out of 50000.0 ps = 24960421/25000000 steps
	Simulating...


In [ ]:
simulations